In [45]:
import sys
sys.path.append("./../")
from evaluation.gsm8k import judge
import json
from thirdparty.openai.grade_school_math.dataset import read_jsonl
import numpy as np

In [46]:
reference = read_jsonl("./../data/gsm8k/test.json")

In [47]:
with open("./../result/output_mistral_instruct_v2_sft_gsm8k.json", "r") as f:
    data = json.load(f)

with open("./../result/output_mistral_instruct_v2_pause_token_sft_gsm8k.json", "r") as f:
    data_with_pause = json.load(f)
    
for entry in data:
    entry["answer"] = entry["output"]

for entry in data_with_pause:
    entry["answer"] = entry["output"]

In [48]:
reference_crop = reference[:len(data)]

In [49]:
no_pause = np.mean(judge(data, reference_crop))
with_pause = np.mean(judge(data_with_pause, reference_crop))

print(no_pause, with_pause)

0.3564356435643564 0.42574257425742573
